# Examples with a Postgres DB

> **These are Go notebooks**: In order to use the GoNB Jupyter Kernel, please install GoNB from here: [https://github.com/janpfeifer/gonb](https://github.com/janpfeifer/gonb)

Note also that for local package development, you can put: `!*go mod edit -replace "github.com/umbralcalc/stochadex=/path/to/stochadex"` at the top of any cell.

## Postgres writing

Writing some stored data directly to a Postgres DB table (see `Dockerfile.postgres` for an example DB setup) is possible with a function from the analysis package. Here's an example of this below using a compound Poisson process with Gamma-distributed jumps.

In [ ]:
import (
	"github.com/umbralcalc/stochadex/pkg/simulator"
	"github.com/umbralcalc/stochadex/pkg/analysis"
	"github.com/umbralcalc/stochadex/pkg/continuous"
	"github.com/umbralcalc/stochadex/pkg/discrete"
)

%%

// Create a simulator.StateTimeStorage from a simulation run
storage := analysis.NewStateTimeStorageFromPartitions(
	// Instantiate the desired simulation state partitions
	[]*simulator.PartitionConfig{
		{
			Name:              "gamma_compound_poisson",
			Iteration:         &continuous.CompoundPoissonProcessIteration{
				JumpDist: &continuous.GammaJumpDistribution{},
			},
			Params:            simulator.NewParams(map[string][]float64{
				"rates": {0.5, 1.0, 0.8},
				"gamma_alphas": {1.0, 2.5, 3.0},
                "gamma_betas": {2.0, 1.0, 4.1},
			}),
			InitStateValues:   []float64{0.0, 0.0, 0.0},
			StateHistoryDepth: 1,
			Seed:              5678,
		},
		{
			Name:              "group_numbers",
			Iteration:         &discrete.BinomialObservationProcessIteration{},
			Params:            simulator.NewParams(map[string][]float64{
				"observed_values": {5, 5, 5},
				"state_value_observation_probs": {0.1, 0.4, 0.7},
				"state_value_observation_indices": {0, 1, 2},
			}),
			InitStateValues:   []float64{0.0, 0.0, 0.0},
			StateHistoryDepth: 1,
			Seed:              321,
		},
    },
	// Decide when should we stop the simulation
	&simulator.NumberOfStepsTerminationCondition{
		MaxNumberOfSteps: 1000,
	},
	// Decide how time should evolve
	&simulator.ConstantTimestepFunction{
		Stepsize: 1.0,
	},
	// Input the initial time
	0.0,
)

// Specify the Postgres DB configuration
db := &analysis.PostgresDb{
	User: "stochadexuser",
	Password: "stochadexpassword",
	Dbname: "stochadexdb",
	TableName: "testnumbers",
}

// Write the data to the configured Postgres DB
analysis.WriteStateTimeStorageToPostgresDb(db, storage)

## Postgres querying

It's also quite easy to query a Postgres DB table for data that can be input into storage. An example of this is below, using the data we wrote to the table from the example above.

In [ ]:
import (
	"math"
	"github.com/umbralcalc/stochadex/pkg/analysis"

	gonb_echarts "github.com/janpfeifer/gonb-echarts"
)

%%

// Specify the Postgres DB configuration
db := &analysis.PostgresDb{
	User: "stochadexuser",
	Password: "stochadexpassword",
	Dbname: "stochadexdb",
	TableName: "testnumbers",
}

// Create a simulator.StateTimeStorage using data from a DB table
storage, _ := analysis.NewStateTimeStorageFromPostgresDb(
	db,
	[]string{"gamma_compound_poisson", "group_numbers"},
	0.0,
	1000.0,
)

// Reference the plotting data for the x-axis
xRef := analysis.DataRef{Plotting: &analysis.DataPlotting{IsTime: true}}

// Reference the plotting data for the y-axis
yRefs := []analysis.DataRef{{PartitionName: "gamma_compound_poisson"}}

// Create a scatter plot from partitions in a simulator.StateTimeStorage
scatter := analysis.NewScatterPlotFromPartition(storage, xRef, yRefs)

// Display the plot in a Go notebook
gonb_echarts.Display(scatter, "width: 1024px; height:400px; background: white;")

## Grouped aggregations

Having the data in the storage format makes it easier to write configurations for partitions which can, e.g., apply grouped aggregation operations. We demonstrate this use case below using data pulled from the Postgres table above.

In [ ]:
import (
	"github.com/umbralcalc/stochadex/pkg/analysis"
	"github.com/umbralcalc/stochadex/pkg/simulator"

	"github.com/go-echarts/go-echarts/v2/opts"
	gonb_echarts "github.com/janpfeifer/gonb-echarts"
)

%%

// Specify the Postgres DB configuration
db := &analysis.PostgresDb{
	User: "stochadexuser",
	Password: "stochadexpassword",
	Dbname: "stochadexdb",
	TableName: "testnumbers",
}

// Create a simulator.StateTimeStorage using data from a DB table
storage, _ := analysis.NewStateTimeStorageFromPostgresDb(
	db,
	[]string{"gamma_compound_poisson", "group_numbers"},
	0.0,
	1000.0,
)

// Apply a grouping to the data in order to get a 'grouped storage'
groupedStorage := analysis.NewGroupedStateTimeStorage(
	analysis.AppliedGrouping{
		GroupBy: []analysis.DataRef{{PartitionName: "group_numbers"}},
		Precision: 1,
	},
	storage,
)

// Get a partition configured for a grouped sum aggregation using the grouped storage
groupedSumPartition := analysis.NewGroupedAggregationPartition(
	general.SumAggregation,
	analysis.AppliedAggregation{
		Name: "grouped_sum",
		Data: analysis.DataRef{PartitionName: "gamma_compound_poisson"},
		DefaultValue: 0,
	},
	groupedStorage,
)

// Run and add the grouped sum aggregation partition to the original storage
storage = analysis.AddPartitionsToStateTimeStorage(
	storage,
	[]*simulator.PartitionConfig{groupedSumPartition},
	map[string]int{"gamma_compound_poisson": 1, "group_numbers": 1},
)

// Reference the plotting data for the x-axis
xRef := analysis.DataRef{Plotting: &analysis.DataPlotting{IsTime: true}}

// Reference the plotting data for the y-axis
yRefs := []analysis.DataRef{{PartitionName: "grouped_sum"}}

// Create a scatter plot from partitions in a simulator.StateTimeStorage
scatter := analysis.NewScatterPlotFromPartition(storage, xRef, yRefs)

// Display the plot in a Go notebook
gonb_echarts.Display(scatter, "width: 1024px; height:400px; background: white;")